27/09/2022

# Add colored segmentation on video

Use this script to add colored annotations and predictions on given dataset movies.

Used to generate examples for midterm exams.

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import datetime
import imageio
import itertools

import numpy as np
import math
from PIL import Image
from scipy import spatial, optimize

from metrics_tools import correspondences_precision_recall, get_sparks_locations_from_mask, process_spark_prediction
from preds_processing_tools import paste_segmentation_on_video, add_colored_annotations_to_video, l2_dist, ball, color_ball

## Load movies, annotations & training predictions

Predictions need to be first saved running `load_trainings_predict.py`.

In [4]:
from dataset_tools import load_annotations_ids,load_movies_ids,load_predictions_ids

In [5]:
# assuming these are movie IDs whose predictions are available...
movie_ids_train = ["01","02","03","04","06","07","08","09",
                            "11","12","13","14","16","17","18","19",
                            "21","22","23","24","27","28","29",
                            "30","33","35","36","38","39",
                            "41","42","43","44","46"]

movie_ids_test  = ["05","10","15","20","25","32","34","40","45"]

movie_ids = movie_ids_train+movie_ids_test

In [6]:
training_name = 'TEMP_new_annotated_peaks_physio'
epoch = 100000

In [7]:
data_dir = os.path.join("..","data","sparks_dataset")

preds_dir = os.path.join("trainings_validation", 
                        training_name)

out_dir = os.path.join("trainings_validation", 
                        training_name, 
                        "colored_segmentation")
os.makedirs(out_dir, exist_ok=True)

In [8]:
### Load movies
xs = load_movies_ids(data_folder=data_dir,
                     ids=movie_ids,
                     names_available = True,
                     movie_names = 'video')

### Load annotations
ys = load_annotations_ids(data_folder=data_dir,
                          ids=movie_ids,
                          mask_names="class_label")

### Load annotated event instances
annotated_events = load_annotations_ids(data_folder=data_dir,
                                        ids=movie_ids,
                                        mask_names="event_label"
                                        )

### Load predictions
# Predictions created from selected model
sparks_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_sparks.tif") for movie_id in movie_ids}
puffs_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_puffs.tif") for movie_id in movie_ids}
waves_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_waves.tif") for movie_id in movie_ids}

sparks = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in sparks_filenames.items()}
puffs = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in puffs_filenames.items()}
waves = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in waves_filenames.items()}

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [23]:
# physiological params
PIXEL_SIZE = 0.2 # 1 pixel = 0.2 um x 0.2 um
global MIN_DIST_XY
MIN_DIST_XY = round(1.8 / PIXEL_SIZE) # min distance in space between sparks
TIME_FRAME = 6.8 # 1 frame = 6.8 ms
global MIN_DIST_T
MIN_DIST_T = round(20 / TIME_FRAME) # min distance in time between sparks

# parameters
ignore_frames = 6

t_detection_sparks = 0.65
min_radius_sparks = 1

transparency = 45

sigma = 1.5

In [25]:
# dataset params
classes = ['sparks', 'puffs', 'waves', 'ignore']
classes_nb = {'sparks': 1,
              'puffs': 3,
              'waves': 2,
              'ignore': 4
             }

## Get predictions segmentation

In [26]:
from preds_processing_tools import get_argmax_segmentation_otsu

In [27]:
preds = {}

In [28]:
for movie_id in movie_ids:
    raw_preds = {'sparks': sparks[movie_id],
                 'puffs': puffs[movie_id],
                 'waves': waves[movie_id],
                 'background': 1-sparks[movie_id]-puffs[movie_id]-waves[movie_id]
                }
    preds[movie_id] = get_argmax_segmentation_otsu(preds=raw_preds,
                                                   get_classes=False,
                                                   debug=False)

## Add segmentation to original movies

In [29]:
from preds_processing_tools import add_colored_segmentation_to_video

### add transparent segmentation on original movies

In [30]:
# visualisation params
color_dict = {'sparks': [178, 255, 102],
              'puffs': [255, 102, 102],
              'waves': [178, 102, 255],
              'ignore': [224, 224, 224]
             }

transparency = 70

In [63]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    # normalize sample movie
    original_movie = 255*(xs[movie_id]/xs[movie_id].max())
    
    colored_preds_video = np.copy(original_movie)
    colored_labels_video = np.copy(original_movie)

    # convert video to RGB
    original_movie_rgb = [Image.fromarray(frame).convert('RGB') 
                          for frame in original_movie]
    colored_preds_video = [Image.fromarray(frame).convert('RGB') 
                           for frame in colored_preds_video]
    colored_labels_video = [Image.fromarray(frame).convert('RGB') 
                            for frame in colored_labels_video]


    for event_class in classes:
        if classes_nb[event_class] in preds[movie_id]:
            binary_preds = preds[movie_id] == classes_nb[event_class]
            colored_preds_video = add_colored_segmentation_to_video(segmentation=binary_preds,
                                                                    video=colored_preds_video,
                                                                    color=color_dict[event_class],
                                                                    transparency=transparency)

        if classes_nb[event_class] in ys[movie_id]:
            binary_labels = ys[movie_id] == classes_nb[event_class]
            colored_labels_video = add_colored_segmentation_to_video(segmentation=binary_labels,
                                                                     video=colored_labels_video,
                                                                     color=color_dict[event_class],
                                                                     transparency=transparency)


    # convert to numpy array and remove first and last frames
    original_movie_rgb = [np.array(frame) 
                          for frame in original_movie_rgb[ignore_frames:-ignore_frames]]
    colored_preds_video = [np.array(frame) 
                           for frame in colored_preds_video[ignore_frames:-ignore_frames]]
    colored_labels_video = [np.array(frame) 
                            for frame in colored_labels_video[ignore_frames:-ignore_frames]]
    
    
    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_original_movie.tif"),
                     original_movie_rgb)

    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_colored_labels.tif"),
                     colored_labels_video)

    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_colored_preds.tif"),
                     colored_preds_video)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video

### create masks with colored segmentation and white background

In [41]:
# visualisation params
color_dict = {'sparks': [0, 255, 0], # green
              'puffs': [220, 20, 60], # red
              'waves': [138, 43, 226], # purple
              'ignore': [80,80,80] # gray
             }

transparency = 1000

In [42]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    # normalize sample movie
    original_movie = 255*(xs[movie_id]/xs[movie_id].max())
    
    # THIS IS DIFFERENT
    colored_preds_video = 255*np.ones_like(original_movie)
    colored_labels_video = 255*np.ones_like(original_movie)

    # convert video to RGB
    original_movie_rgb = [Image.fromarray(frame).convert('RGB') 
                          for frame in original_movie]
    colored_preds_video = [Image.fromarray(frame).convert('RGB') 
                           for frame in colored_preds_video]
    colored_labels_video = [Image.fromarray(frame).convert('RGB') 
                            for frame in colored_labels_video]


    for event_class in classes:
        if classes_nb[event_class] in preds[movie_id]:
            binary_preds = preds[movie_id] == classes_nb[event_class]
            colored_preds_video = add_colored_segmentation_to_video(segmentation=binary_preds,
                                                                    video=colored_preds_video,
                                                                    color=color_dict[event_class],
                                                                    transparency=transparency)

        if classes_nb[event_class] in ys[movie_id]:
            binary_labels = ys[movie_id] == classes_nb[event_class]
            colored_labels_video = add_colored_segmentation_to_video(segmentation=binary_labels,
                                                                     video=colored_labels_video,
                                                                     color=color_dict[event_class],
                                                                     transparency=transparency)


    # convert to numpy array and remove first and last frames
    original_movie_rgb = [np.array(frame) 
                          for frame in original_movie_rgb[ignore_frames:-ignore_frames]]
    colored_preds_video = [np.array(frame) 
                           for frame in colored_preds_video[ignore_frames:-ignore_frames]]
    colored_labels_video = [np.array(frame) 
                            for frame in colored_labels_video[ignore_frames:-ignore_frames]]
    
    
    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_original_movie.tif"),
                     original_movie_rgb)

    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_colored_labels_white_background.tif"),
                     colored_labels_video)

    imageio.volwrite(os.path.join(out_dir, 
                                  training_name+"_"+str(epoch)+"_"+movie_id+"_colored_preds_white_background.tif"),
                     colored_preds_video)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video